In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os
%load_ext autoreload
%autoreload 2
print('TF version:', tf.__version__ , '\nGPU available:', tf.test.is_gpu_available())

TF version: 2.0.0 
GPU available: True


# MovieLens Dataset

In [2]:
path = '../'
data_path = 'datasets/' # Paperspace
file_name = 'am_like_ml_01'

In [3]:
df = pd.read_pickle(path + data_path + file_name)[:20000]
df['item_id'] = df.item.astype('category').cat.codes
df['user_id'] = df.user.astype('category').cat.codes
df.head()

,user_id,user,item,datetime,rating,item_id
0,0,A0039616ADOZ0KMWQRNX,B00QFJG1U8,2016-10-04,5.0,10163
1,0,A0039616ADOZ0KMWQRNX,B010ACF2PK,2016-10-04,5.0,11999
2,0,A0039616ADOZ0KMWQRNX,B00BFE0IZ2,2016-10-04,5.0,5441
3,0,A0039616ADOZ0KMWQRNX,B01CZMQCPC,2016-10-04,5.0,14386
4,0,A0039616ADOZ0KMWQRNX,B01B5DLG7G,2016-10-26,4.0,14047


# Init Params

In [4]:
res_ext = file_name[:2]
total_items = len(df.item_id.unique())
first_model_id = str(0) + '_' + res_ext

In [5]:
# all_models = pd.read_pickle(path + 'results/' + res_ext + '/all_models')
# new_model_id = str(int(all_models.model_id.max()[0]) + 1) + '_' + res_ext

In [6]:
params = {
'model_id':first_model_id,
'train_time':0,
'epochs':0,

# Grid Search params
'BATCH_SIZE':32,
'learning_rate':0.1,
'delta':0.2,             # Diversity Bias
'max_seq_len':30,        # Max length of sequence71=median
'embedding_dim':100,
'rnn_units':20,
    
'val_perc':0.1,          # Percentage of users from df in val and test set
'test_perc':0.1, 
'n_items_val':0,        # Number of last (chronologically) items in val and test set
'n_items_test':1,

'pad_value':total_items, # Pad with total_items+1 => masked => still use item 0
'shift_targets_by':1  
}

In [7]:
BATCH_SIZE = params['BATCH_SIZE']
learning_rate = params['learning_rate']
delta = params['delta']
max_seq_len = params['max_seq_len']

val_perc = params['val_perc']
test_perc = params['test_perc']
n_items_val = params['n_items_val']
n_items_test = params['n_items_test']

pad_value = params['pad_value']
shift_targets_by = params['shift_targets_by'] 

embedding_dim = params['embedding_dim']
rnn_units = params['rnn_units']

# Imports

In [8]:
from Data_prep import train_val_test_split, create_seq_batch_dataset
from Models import build_LSTM_model, store_LSTM_model
from Evaluation import recall_metric, diversity_bias_loss, create_diversity_bias, get_predictions, get_metrics
from Helpers import TimingCallback

# Init Datasets

In [9]:
# Train Test Val Split
data_split = train_val_test_split(df, val_perc, test_perc, n_items_val, n_items_test, seqs=True)
train_set, val_set, val_left_out_items, test_set, test_left_out_items = data_split 

# Configure Checkpoint directory

In [10]:
ext = '_' + file_name[:2] #ML or Am
# directory = './ckpts/ckpts' 
directory = '../ckpts/ckpts'

---
# Grid Search

In [11]:
learning_rates = [0.01, 0.1, 0.5]
epochs = [2]# [100, 400]
deltas = [0.2, 0.6]
batch_sizes = [32, 64]
max_seq_lens = [18, 30] #Median=18
rnn_units = [20, 50]

total_runs = len(learning_rates) * len(epochs) * len(deltas) * len(batch_sizes) * len(max_seq_lens) * len(rnn_units)
runs = 0
print('Total runs:', total_runs)

Total runs: 48


In [14]:
metrics=[recall_metric(total_items=total_items)]
for epoch in epochs:
    params['epochs'] = epoch
    for delta in deltas:
        params['delta'] = delta
        
        # Pre-Calculate diversity_bias
        diversity_bias = create_diversity_bias(train_set, total_items, delta)
        loss=diversity_bias_loss(db=diversity_bias, total_items=total_items)
        
        for rnn_unit in rnn_units:
            params['rnn_units'] = rnn_unit
        
            for batch_size in batch_sizes:
                params['BATCH_SIZE'] = batch_size
                
                # Rebuild model 
                model = build_LSTM_model(total_items = total_items, embedding_dim = embedding_dim, mask_value = pad_value, rnn_units = rnn_unit, batch_size = batch_size, return_sequences=True)

                for max_seq_len in max_seq_lens:
                    params['max_seq_len'] = max_seq_len

                    # Create new datasets
                    train_dataset = create_seq_batch_dataset(df=train_set, shift=shift_targets_by, max_seq_len=max_seq_len, pad_value=pad_value, batch_size=batch_size, stats=False, drop_remainder=True)
                    val_dataset = create_seq_batch_dataset(df=val_set, shift=shift_targets_by, max_seq_len=max_seq_len, pad_value=pad_value, batch_size=batch_size, stats=False, drop_remainder=True) 

                    for learning_rate in learning_rates:
                        runs += 1

                        # Print current run
                        print('='*50, '\nRun:', str(runs) + '/' + str(total_runs))
                        print('Number of epochs:\t', epoch)
                        print('Number of (h) units:\t', rnn_unit)
                        print('Batch size:\t\t', batch_size)
                        print('Max seq len:\t\t', max_seq_len)
                        print('Learning rate:\t\t', learning_rate)

                        params['learning_rate'] =  learning_rate

                        # Compile Model
                        model = build_LSTM_model(total_items = total_items, embedding_dim = embedding_dim, mask_value = pad_value, rnn_units = rnn_unit, batch_size = batch_size, return_sequences=True)
                        optimizer=tf.keras.optimizers.Adagrad(lr=learning_rate)

                        model.compile(optimizer=optimizer,
                                      loss=loss, 
                                      metrics=metrics)

                        # Create Callbacks
                        checkpoint_dir = directory + '_' + str(params['model_id'])
                        checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
                        
                        checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath = checkpoint_prefix, monitor = 'val_recall', mode = 'max', save_best_only = True, save_weights_only = True)
                        early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor = 'val_recall', min_delta = 0.0001, mode = 'max', patience = 15)
                        timing_callback = TimingCallback()
                        callbacks = [checkpoint_callback, early_stopping_callback, timing_callback]
                        
                        # Run 
                        history = model.fit(x=train_dataset, 
                                            validation_data=val_dataset, 
                                            epochs=epoch,
                                            callbacks=callbacks,
                                            verbose=1)
                        
                        
                        # Restore lates checkpoint and predict 
                        model = build_LSTM_model(total_items = total_items, embedding_dim = embedding_dim, mask_value = pad_value, rnn_units = rnn_unit, batch_size = 1, return_sequences=False)
                        model.load_weights(tf.train.latest_checkpoint(checkpoint_dir)).expect_partial()
                        model.build(tf.TensorShape([1, None]))
                        preds_df = get_predictions(model, test_set, test_left_out_items, 20)
                        metrics_test = get_metrics(preds_df, 5, 20)
                        print(metrics_test)
                        
                        # Store model
                        store_path = path + 'results/' + res_ext + '/all_models'
                        train_time = np.sum(timing_callback.logs)
                        all_models = store_LSTM_model(store_path, params.copy(), history.history.copy(), train_time, metrics_test, store=True)
                        
                        # Change Model Id for next model
                        params['model_id'] = str(int(params['model_id'][0]) + 1) + '_am'


Run: 7/48
Number of epochs:	 2
Number of (h) units:	 20
Batch size:		 32
Max seq len:		 18
Learning rate:		 0.01
Epoch 1/2
23/23 [==============================] - 9s 391ms/step - loss: 1.6894 - recall: 0.0000e+00 - val_loss: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/2
23/23 [==============================] - 0s 19ms/step - loss: 1.6854 - recall: 0.0000e+00 - val_loss: 6.8572 - val_recall: 0.0000e+00
Obtaining metrics time: 0.11
   rank_at  hitcounts  recall  precision
0        1          0     0.0        0.0
1        5          0     0.0        0.0
2       10          0     0.0        0.0
3       15          0     0.0        0.0
4       20          0     0.0        0.0
Run: 8/48
Number of epochs:	 2
Number of (h) units:	 20
Batch size:		 32
Max seq len:		 18
Learning rate:		 0.1
Epoch 1/2
23/23 [==============================] - 9s 394ms/step - loss: 1.6892 - recall: 0.0000e+00 - val_loss: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/2
23/23 [==============================] - 0s 18ms

KeyboardInterrupt: 